## Introduction

In this assignment, you are going to build a classifier for named entities from the Groningen Meaning Bank corpus.  Named entity recognition (NER) takes noun phrases from a text and identifies whether they are persons, organizations, and so on.  You will be using the Groningen Meaning Bank named entity corpus available on mltgpu at `/scratch/lt2222-v21-resources/GMB_dataset.txt`.  In this version of the task, you will assume we know *that* something is a named entity, and instead use multi-class classification to identify its type.  So you will be doing named entity classification but *not* recognition.

The data looks like this: 

```
3996    182.0   Nicole  NNP     B-per
3997    182.0   Ritchie NNP     I-per
3998    182.0   is      VBZ     O
3999    182.0   pregnant        JJ      O
4000    182.0   .       .       O
4001    183.0   Speaking        VBG     O
4002    183.0   to      TO      O
4003    183.0   ABC     NNP     B-org
4004    183.0   News    NNP     I-org
4005    183.0   interviewer     NN      O
4006    183.0   Dianne  NNP     B-per
4007    183.0   Sawyer  NNP     I-per
4008    183.0   ,       ,       O
4009    183.0   the     DT      O
4010    183.0   25-year-old     JJ      O
4011    183.0   co-star NN      O
4012    183.0   of      IN      O
4013    183.0   TV      NN      O
4014    183.0   's      POS     O
4015    183.0   The     DT      B-art
4016    183.0   Simple  NNP     I-art
4017    183.0   Life    NNP     I-art
4018    183.0   said    VBD     O
4019    183.0   she     PRP     O
4020    183.0   is      VBZ     O
4021    183.0   almost  RB      O
4022    183.0   four    CD      O
4023    183.0   months  NNS     O
4024    183.0   along   IN      O
4025    183.0   in      IN      O
4026    183.0   her     PRP$    O
4027    183.0   pregnancy       NN      O
4028    183.0   .       .       O
```

The first column is the line number.  The second column is a sentence number (for some reason given as a float; ignore it).  The third column is the word.  The fourth column is a part of speech (POS) tag in Penn Treebank format.  The last column contains the named entity annotation. 

The annotation works like this.  Every `O` just means that the row does not represent a named entity.  `B-xyx` means the first word in a named entity with type `xyx`. `I-xyz` means the second and later words of an `xyz` entity, if there are any.  That means that every time there's a `B` or an `I`, there's a named entity.  

The entity types in the corpus are `art`,
`eve`,
`geo`,
`gpe`,
`nat`,
`org`,
`per`,
and `tim`

Your task is the following.
1. To preprocess the text (lowercase and lemmatize; punctuation can be preserved as it gets its own rows).
2. To create instances from every from every identified named entity in the text with the type of the NE as the class, and a surrounding context of five words on either side as the features.  
3. To generate vectors and split the instances into training and testing datasets at random.
4. To train a support vector machine (via `sklearn.svm.LinearSVC`) for classifying the NERs.
5. To evaluate the performance of the classifier.

You will do this by modifying a separate file containing functions that will be called from this notebook as a module.  You can modify this notebook for testing purposes but please only submit the original.  You will document everything in Markdown in README.md and submit a GitHub repository URL.

This assignment is due on **Tuesday, 2021 March 9 at 23:59**.  It has **25 points** and **7 bonus points**.

In [47]:
import a2
from sklearn.svm import LinearSVC

In [48]:
gmbfile = open('/home/xsayas@GU.GU.SE/scratch/lt2222-v21-resources/GMB_dataset.txt', "r")

## Part 1 - preprocessing (3 points)

See step 1 above.  The data is coming to you as an unused file handle object.  You can return the data in any indexable form you like.  You can also choose to remove infrequent or uninformative words to reduce the size of the feature space. (Document this in README.md.)

In [49]:
import sys
import os
import numpy as np
import numpy.random as npr
import pandas as pd
import random
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import string
import re



def preprocess(inputfile):
    lemmatizer = WordNetLemmatizer()
    ps = PorterStemmer()
    contents = [x.strip().split("\t") for x in inputfile.readlines()]
    #Filter out remaining items that are not alphabetic.
    c_contents = [l for l in contents if l[2].isalpha()]
    #''.join([i for i in raw_file if not i.isdigit()])

    
    rows = []
    for row in c_contents[1:]:
        row[2] = ps.stem(row[2])
        row[2] = lemmatizer.lemmatize(row[2].lower())
        row[2] = re.sub(r'[^\w\s]','',row[2])
        rows.append(row)
    
    clean_rows = []    
    for line in rows:
        if (line[2] != ""):
            clean_rows.append(line)
            

                 
    col = ['WordN', 'SentN', 'Word', 'Pos', 'Type']
    return pd.DataFrame(clean_rows, columns = col)


 
#preprocess(gmbfile)   



In [50]:
inputdata = preprocess(gmbfile)

inputdata[20:40]

,WordN,SentN,Word,Pos,Type
20,20,1.0,from,IN,O
21,21,1.0,that,DT,O
22,22,1.0,countri,NN,O
23,24,2.0,famili,NNS,O
24,25,2.0,of,IN,O
25,26,2.0,soldier,NNS,O
26,27,2.0,kill,VBN,O
27,28,2.0,in,IN,O
28,29,2.0,the,DT,O
29,30,2.0,conflict,NN,O


## Part 2 - Creating instances (7 points)

Do step 2 above.  You will create a collection of Instance objects.  Remember to consider the case where the NE is at the beginning of a sentence or at the end, or close to either (you can create a special start token for that).  You can also start counting from before the `B` end of the NE mention and after the last `I` of the NE mention. That means that the instances should include things before and after the named entity mention, but not the named entity text itself.

In [51]:
import re
class Instance:
    def __init__(self, neclass, features):
        self.neclass = neclass
        self.features = features

    def __str__(self):
        return "Class: {} Features: {}".format(self.neclass, self.features)

    def __repr__(self):
        return str(self)



def create_instances(data):
    instances = []
    sent_num = list(data['SentN'])
    word = list(data['Word'])
    type_ent = list(data['Type'])
    tuples_ent = list(zip(sent_num, word, type_ent))
    for index, tup in enumerate(tuples_ent):
        features = []
        last_features = []
        if tup[2].startswith('B'):
            neclass = re.sub('B-', '', tup[2])
            for i in reversed(list(range(1, 6))): #the 5 freatures before the word with a NE
                if index-i > 0:
                    if tuples_ent[index-i][0] == tup[0]:
               
                        features.append(tuples_ent[index-i][1]) #Append the word
                    else:
                        features.append("<s>")
                else:
                    features.append("<s>")
            counter = 1
            while tuples_ent[index+counter][2].startswith('I'):
                counter += 1

            for j in range(1, 6):#the 5 freatures after the word with a NE
                if j+index+counter < len(tuples_ent):
                    if tuples_ent[index+j+counter][0] == tup[0]:
          
                        last_features.append(tuples_ent[index+j+counter][1]) #Append the word
                    else:
                        last_features.append("<e>")

                    
                else:
                    last_features.append("<e>")

            features.extend(last_features) 
            instances.append(Instance(neclass, features))
    return instances
    
      
        

In [52]:
instances = create_instances(inputdata)

In [53]:
instances[20:40]

[Class: gpe Features: ['<s>', '<s>', '<s>', '<s>', '<s>', 'week', 'restart', 'part', 'of', 'the'],
 Class: geo Features: ['the', 'convers', 'process', 'at', 'it', 'plant', '<e>', '<e>', '<e>', '<e>'],
 Class: gpe Features: ['<s>', '<s>', '<s>', '<s>', '<s>', 'say', 'they', 'expect', 'to', 'get'],
 Class: tim Features: ['sensit', 'part', 'of', 'the', 'plant', 'an', 'iaea', 'surveil', 'system', 'begin'],
 Class: org Features: ['the', 'plant', 'wednesday', 'after', 'an', 'system', 'begin', 'function', '<e>', '<e>'],
 Class: org Features: ['<s>', '<s>', '<s>', '<s>', 'the', 'back', 'ha', 'threaten', 'to', 'refer'],
 Class: gpe Features: ['back', 'ha', 'threaten', 'to', 'refer', 'the', 'secur', 'council', 'which', 'could'],
 Class: gpe Features: ['impos', 'sanction', 'if', 'it', 'find', 'violat', 'the', 'nuclear', 'treati', '<e>'],
 Class: art Features: ['find', 'tehran', 'ha', 'violat', 'the', '<e>', '<e>', '<e>', '<e>', '<e>'],
 Class: gpe Features: ['<s>', '<s>', '<s>', '<s>', '<s>', 'pr

## Part 3 - Creating the table and splitting (10 points)

Here you're going to write the functions that create a data table with "document" vectors representing each instance and split the table into training and testing sets and random with an 80%/20% train/test split.

In [54]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD

def create_table(instances):
    features_items = []
    for instance in instances:
        for feature in instance.features:
            if feature not in features_items:
                features_items.append(feature)
    data = []
    for instance in instances:
        for feature in instance.features:
            data += [[instance.neclass] + [feature.count(word) for word in features_items]]
    features_items = ["class"] + features_items
    df = pd.DataFrame(data, columns = features_items)
    red_df = reduce(df)
    return red_df

def reduce(matrix, dims=300):
    data= matrix.copy()
    df_x = data.iloc[:,1:] #full matrix without the first column
    df_y = data.iloc[:,0]  # file name column
 
    col_names = df_x.columns.tolist() #here I put the name of column in the df_x
    feature_names = col_names
  
    svd = TruncatedSVD(n_components=dims, n_iter=7, random_state=42)
    svd = svd.fit(df_x)

    mat_r = svd.transform(df_x)
    
  
    return pd.concat([pd.DataFrame(df_y), pd.DataFrame(mat_r)], axis=1)


In [55]:
bigdf = create_table(instances)
bigdf[20:40]

,class,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
20,gpe,9.810977e-01,-7.351802e-01,-3.306643e-02,1.996402e-16,1.477021e-15,-1.705935e-01,1.031342e+00,7.013426e-02,1.587551e-02,...,1.659244e-03,1.778998e-03,2.190109e-04,9.993522e-04,7.586987e-05,-1.269055e-03,1.779343e-03,-1.598995e-03,1.376567e-03,1.165040e-03
21,gpe,1.014149e+00,-7.499153e-01,-3.324420e-02,2.183136e-16,1.389957e-15,-1.655076e-01,1.055408e+00,6.654286e-02,5.783781e-02,...,-5.154849e-02,-6.775390e-03,-1.622951e-02,6.999836e-03,3.344678e-02,2.567834e-02,-3.995172e-03,5.470610e-02,-1.910995e-02,4.508251e-04
22,gpe,1.684134e-02,-2.755181e-03,1.404772e+00,2.250223e-16,-2.042569e-16,-9.300912e-02,1.532277e-02,1.540302e-03,-2.878203e-02,...,3.487224e-04,2.821485e-05,-1.566503e-04,2.193644e-04,-1.977974e-04,2.817326e-04,-3.480564e-04,2.488501e-04,-3.894884e-04,-3.342777e-04
23,gpe,1.400539e+00,3.455274e-01,-2.780728e-02,2.131547e-16,1.534186e-15,-1.404460e-01,-1.520117e-01,-5.932189e-01,-6.853158e-02,...,9.921250e-03,-1.495884e-02,-8.696044e-03,3.750023e-03,-6.911614e-03,1.019024e-02,-4.321206e-03,2.042180e-03,-6.069935e-03,2.243094e-02
24,gpe,6.730089e-03,1.795319e-02,-4.459281e-04,4.870776e-17,8.349337e-17,-4.894785e-03,2.931370e-03,-5.501915e-02,-1.296474e-03,...,4.204609e-04,4.089851e-04,5.052212e-04,-1.982593e-04,2.442355e-04,-3.491109e-04,9.798896e-04,3.326819e-05,1.541762e-04,1.081597e-05
25,gpe,-6.094439e-18,-4.055769e-19,5.673238e-19,2.808557e-18,2.206705e-18,1.652348e-17,-2.847550e-17,-8.287639e-18,-4.140686e-18,...,1.938315e-10,-6.228469e-10,-1.807041e-10,-7.193239e-11,3.776117e-10,-7.624682e-10,5.849917e-10,-8.786831e-11,5.372741e-10,2.071783e-10
26,gpe,8.910410e-01,-6.833888e-01,-1.978865e-02,1.384124e-16,1.182157e-15,-1.303700e-01,-4.588184e-01,5.893151e-03,-7.002902e-02,...,2.469109e-04,7.186670e-04,-6.195324e-04,-1.227571e-03,8.908869e-04,1.015357e-03,3.450118e-04,4.151323e-04,-6.860050e-04,-1.180726e-03
27,gpe,6.792235e-01,7.894062e-01,5.286455e-02,-1.066667e-15,-8.942696e-15,9.679079e-01,7.427102e-02,-1.535182e-01,-5.555154e-02,...,-8.314051e-03,8.966059e-03,-6.461752e-03,1.803923e-03,6.648608e-03,-1.047502e-02,8.130200e-03,3.224700e-03,8.768805e-03,1.805857e-03
28,gpe,-2.651827e-20,7.778454e-18,-3.814026e-17,1.000000e+00,-6.938404e-15,8.798167e-16,3.796000e-17,-9.666479e-17,1.640503e-17,...,1.462402e-17,2.841301e-18,-3.058962e-18,-1.308596e-17,-2.031964e-17,-1.014928e-18,-1.308476e-17,-2.912364e-17,-5.577363e-19,-1.198421e-17
29,gpe,-2.651827e-20,7.778454e-18,-3.814026e-17,1.000000e+00,-6.938404e-15,8.798167e-16,3.796000e-17,-9.666479e-17,1.640503e-17,...,1.462402e-17,2.841301e-18,-3.058962e-18,-1.308596e-17,-2.031964e-17,-1.014928e-18,-1.308476e-17,-2.912364e-17,-5.577363e-19,-1.198421e-17


In [56]:


def ttsplit(bigdf):
    train = bigdf.sample(frac=0.8)
    df_train = train.reset_index()
    df_test = bigdf.drop(train.index).reset_index()
    
    
       
    return df_train.drop('class', axis=1).to_numpy(), df_train['class'], df_test.drop('class', axis=1).to_numpy(), df_test['class']

In [57]:
train_X, train_y, test_X, test_y = ttsplit(bigdf)

# X and y mean feature matrix and class respectively.
train_X, train_y, test_X, test_y

(array([[ 3.22100000e+04,  1.85784901e-20,  3.78758374e-21, ...,
         -6.18315141e-03, -7.07238661e-03,  3.58183382e-03],
        [ 3.58520000e+04,  1.17942469e-02, -3.15035525e-03, ...,
         -1.54410773e-03,  6.63544179e-03,  2.89353629e-03],
        [ 6.23000000e+02,  1.68515045e-02, -2.75735343e-03, ...,
          1.41482706e-03, -1.87125479e-03, -3.57700389e-04],
        ...,
        [ 1.44700000e+03, -2.65182665e-20,  7.77845405e-18, ...,
         -2.91236377e-17, -5.57736272e-19, -1.19842131e-17],
        [ 2.63520000e+04,  7.11893724e-18,  1.63153360e-17, ...,
         -7.82167698e-18,  2.32459300e-17,  1.62447620e-17],
        [ 3.35830000e+04,  9.57079819e-03,  7.50727451e-03, ...,
          3.42620988e-03,  1.32484100e-03,  6.13382610e-04]]), 0        tim
 1        geo
 2        geo
 3        gpe
 4        geo
         ... 
 48427    geo
 48428    tim
 48429    org
 48430    geo
 48431    per
 Name: class, Length: 48432, dtype: object, array([[ 5.00000000e+00,  4.3038

In [58]:
len(test_y) / (len(test_y) + len(train_y))

0.2

In [59]:
len(test_X) / (len(test_X) + len(train_X))

0.2

In [60]:
test_y[0]

'geo'

## Part 4 - Training the model (0 points)

This part you won't do yourself.

In [61]:
model = LinearSVC()
model.fit(train_X, train_y)
train_predictions = model.predict(train_X)
test_predictions = model.predict(test_X)

/usr/local/lib64/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [62]:
train_predictions

array(['per', 'per', 'geo', ..., 'geo', 'per', 'per'], dtype=object)

In [63]:
train_y

0        tim
1        geo
2        geo
3        gpe
4        geo
        ... 
48427    geo
48428    tim
48429    org
48430    geo
48431    per
Name: class, Length: 48432, dtype: object

In [64]:
test_predictions

array(['geo', 'geo', 'geo', ..., 'per', 'per', 'per'], dtype=object)

In [65]:
test_y

0        geo
1        geo
2        geo
3        geo
4        geo
        ... 
12103    per
12104    per
12105    tim
12106    tim
12107    tim
Name: class, Length: 12108, dtype: object

## Part 5 - Evaluation (5 points)

Investigate for yourself what a "confusion matrix".  Then implement a function that takes the data and produces a confusion matrix in any readable form that allows us to compare the performance of the model by class.  

In [66]:
from sklearn.metrics import confusion_matrix as cm
def confusion_matrix(truth, predictions):
    labels = ['art','eve','geo','gpe','nat','org','per','tim']
    matrix = cm(truth, predictions, labels = labels )
    df = pd.DataFrame(matrix, index = labels, columns = labels)
    return df
    
    
    


In [67]:
confusion_matrix(test_y, test_predictions)

,art,eve,geo,gpe,nat,org,per,tim
art,0,0,1,0,0,0,95,0
eve,0,0,1,0,0,0,93,0
geo,0,0,147,7,0,0,3625,0
gpe,0,0,95,5,0,0,2217,0
nat,0,0,0,0,0,0,31,0
org,0,0,112,5,0,0,2098,0
per,0,0,42,2,0,0,1726,0
tim,0,0,37,1,0,0,1768,0


In [68]:
confusion_matrix(train_y, train_predictions)

,art,eve,geo,gpe,nat,org,per,tim
art,0,0,24,2,0,0,368,0
eve,0,0,0,0,0,0,326,0
geo,0,0,670,15,0,3,14633,0
gpe,0,0,447,19,0,0,8857,0
nat,0,0,0,0,0,0,109,0
org,0,0,456,14,0,2,7993,0
per,0,0,226,6,0,0,6828,0
tim,0,0,240,5,0,1,7188,0


Examine the matrix and describe your observations in README.md.  In particular, what do you notice about the predictions on the training data compared to those on the test data.

## Bonus Part A - Error analysis (2 points)

Look at the weakest-performing classes in the confusion matrix (or any, if they all perform poorly to the same extent).  Find some examples in the test data on which the classifier classified incorrectly for those classes.  What do you think is the reason why those are hard?  Consider linguistic factors and statistical factors, if applicable.  Write your answer in README.md.

## Bonus Part B - Expanding the feature space (7 points)

Run the entire process above, but incorporate part-of-speech tag information into the feature vectors.  It's your choice as to how to do this, but document it in README.md.  Your new process should run from the single call below:

In [18]:
a2.bonusb('/scratch/lt2222-v21-resources/GMB_dataset.txt')